In [3]:
#input the info needed to create a connection between sql server and this notebook using sqlalchemy
import getpass
mypasswd = getpass.getpass()
username = 'sebmb'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

········


In [4]:
#create a connection and delete my password once connected
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}

engine = create_engine(URL(**postgres_db), echo=True)

del mypasswd

In [2]:
#load in the file and clean up the data into a pandas dataframe
import pandas as pd

file = 'SpotifyTop50.csv'

spot_df = pd.read_csv(file,  encoding='latin-1')
spot_df.columns = [x.lower() for x in spot_df.columns]
spot_df = spot_df.rename(columns={spot_df.columns[0] : 'id', 'beats.per.minute' : 'bpm', 'loudness..db..' : 'loudness', 
                                 'acousticness..' : 'acousticness', 'speechiness.' : 'speechiness', 
                                 'valence.' : 'positivity', 'length.' : 'length', 'track.name' : 'track',
                                 'artist.name' : 'artist'})
spot_df = spot_df.dropna()
spot_df.head()

,id,track,artist,genre,bpm,energy,danceability,loudness,liveness,positivity,length,acousticness,speechiness,popularity
0,1,Senorita,Shawn Mendes,canadian pop,117,55,76,-6,8,75,191,4,3,79
1,2,China,Anuel AA,reggaeton flow,105,81,79,-4,8,61,302,8,9,92
2,3,boyfriend (with Social House),Ariana Grande,dance pop,190,80,40,-4,16,70,186,12,46,85
3,4,Beautiful People (feat. Khalid),Ed Sheeran,pop,93,65,64,-8,8,55,198,12,19,86
4,5,Goodbyes (Feat. Young Thug),Post Malone,dfw rap,150,65,58,-4,11,18,175,45,7,94


In [5]:
#move the csv file from pandas to sql to query the results in the next few cells
spot_df.to_sql('spot_df', # The table to load
          engine,             # The engine created above
          schema= username,   # The schema where the table lives, our pawprint
          if_exists='replace', # If the table is found, it would keep loading the end of table.
          index=False,        # Recall that panda data frame has a row index, so we need to ignore it
          chunksize=50) 

2021-12-27 15:05:07,873 INFO sqlalchemy.engine.base.Engine select version()
2021-12-27 15:05:07,874 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 15:05:07,876 INFO sqlalchemy.engine.base.Engine select current_schema()
2021-12-27 15:05:07,877 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 15:05:07,879 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-12-27 15:05:07,879 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 15:05:07,882 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-12-27 15:05:07,882 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 15:05:07,883 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2021-12-27 15:05:07,884 INFO sqlalchemy.engine.base.Engine {}
2021-12-27 15:05:07,886 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2021-12-27 15:05:07

Below this cell is code copy and pasted from my personal server through Mizzou's SQL server. I give samples of different queries that can be done using the given datatable. 

This cell is a preview of what the data table looks like in sql


SELECT * FROM sebmb.spot_df LIMIT 2;

 id |  track   |    artist    |     genre      | bpm | energy | danceability | loudness | liveness | positivity | length | ac
ousticness | speechiness | popularity
----+----------+--------------+----------------+-----+--------+--------------+----------+----------+------------+--------+---
-----------+-------------+------------
  1 | Senorita | Shawn Mendes | canadian pop   | 117 |     55 |           76 |       -6 |        8 |         75 |    191 |
         4 |           3 |         79
  2 | China    | Anuel AA     | reggaeton flow | 105 |     81 |           79 |       -4 |        8 |         61 |    302 |
         8 |           9 |         92
(2 rows)

This query is the number of songs that made the top 50 list of 2019 by genre. Dance pop and pop were the top genres listened to.

SELECT genre, COUNT(*) FROM sebmb.spot_df GROUP BY genre ORDER BY COUNT(*) DESC;

      genre       | count
------------------+-------
 dance pop        |     8
 pop              |     7
 latin            |     5
 canadian hip hop |     3
 edm              |     3
 reggaeton        |     2
 dfw rap          |     2
 country rap      |     2
 brostep          |     2
 canadian pop     |     2
 panamanian pop   |     2
 reggaeton flow   |     2
 electropop       |     2
 pop house        |     1
 boy band         |     1
 australian pop   |     1
 trap music       |     1
 atl hip hop      |     1
 big room         |     1
 escape room      |     1
 r&b en espanol   |     1
(21 rows)

These are the artists that made the list. Ed Sheeran made the top 50 the most with 4 songs. 

SELECT artist, COUNT(*) FROM sebmb.spot_df GROUP BY artist ORDER BY COUNT(*) DESC;

      artist      | count
------------------+-------
 Ed Sheeran       |     4
 Shawn Mendes     |     2
 Marshmello       |     2
 J Balvin         |     2
 Post Malone      |     2
 Billie Eilish    |     2
 Ariana Grande    |     2
 The Chainsmokers |     2
 Lil Nas X        |     2
 Sech             |     2
 ROSALÍA          |     1
 MEDUZA           |     1
 Ali Gatie        |     1
 Jhay Cortez      |     1
 Taylor Swift     |     1
 Lizzo            |     1
 Lewis Capaldi    |     1
 Drake            |     1
 Anuel AA         |     1
 Maluma           |     1
 Lunay            |     1
 Bad Bunny        |     1
 Daddy Yankee     |     1
 Chris Brown      |     1
 Katy Perry       |     1
 Sam Smith        |     1
 Khalid           |     1
 Jonas Brothers   |     1
 Martin Garrix    |     1
 Lauv             |     1
 Lady Gaga        |     1
 Y2K              |     1
 Tones and I      |     1
 Nicky Jam        |     1
 Kygo             |     1
 Young Thug       |     1
 DJ Snake         |     1
 Lil Tecca        |     1
(38 rows)

This query takes the track name, artist and beats per minute for each song that has a bpm over 100. 

SELECT track,artist,bpm FROM sebmb.spot_df WHERE bpm >= 100 ORDER BY bpm DESC;

                             track                              |      artist      | bpm
----------------------------------------------------------------+------------------+-----
 boyfriend (with Social House)                                  | Ariana Grande    | 190
 Ransom                                                         | Lil Tecca        | 180
 Otro Trago - Remix                                             | Sech             | 176
 Te Robaré                                                      | Nicky Jam        | 176
 LA CANCIÓN                                                     | J Balvin         | 176
 Otro Trago                                                     | Sech             | 176
 Callaita                                                       | Bad Bunny        | 176
 Truth Hurts                                                    | Lizzo            | 158
 Panini                                                         | Lil Nas X        | 154
 Antisocial (with Travis Scott)                                 | Ed Sheeran       | 152
 Goodbyes (Feat. Young Thug)                                    | Post Malone      | 150
 7 rings                                                        | Ariana Grande    | 140
 Sucker                                                         | Jonas Brothers   | 138
 Old Town Road - Remix                                          | Lil Nas X        | 136
 Talk                                                           | Khalid           | 136
 bad guy (with Justin Bieber)                                   | Billie Eilish    | 135
 bad guy                                                        | Billie Eilish    | 135
 Lalala                                                         | Y2K              | 130
 If I Can't Have You                                            | Shawn Mendes     | 124
 Piece Of Your Heart                                            | MEDUZA           | 124
 Senorita                                                       | Shawn Mendes     | 117
 Summer Days (feat. Macklemore & Patrick Stump of Fall Out Boy) | Martin Garrix    | 114
 How Do You Sleep?                                              | Sam Smith        | 111
 Someone You Loved                                              | Lewis Capaldi    | 110
 China                                                          | Anuel AA         | 105
 Call You Mine                                                  | The Chainsmokers | 104
 Higher Love                                                    | Kygo             | 104
 I Don't Care (with Justin Bieber)                              | Ed Sheeran       | 102
 Money In The Grave (Drake ft. Rick Ross)                       | Drake            | 101
 Happier                                                        | Marshmello       | 100
 Never Really Over                                              | Katy Perry       | 100
(31 rows)

This query displays the track name and artist that made the top list in the dance pop genre. 

SELECT track,artist FROM sebmb.spot_df WHERE genre = 'dance pop';

                                     track                                      |    artist
--------------------------------------------------------------------------------+---------------
 boyfriend (with Social House)                                                  | Ariana Grande
 Loco Contigo (feat. J. Balvin & Tyga)                                          | DJ Snake
 No Guidance (feat. Drake)                                                      | Chris Brown
 7 rings                                                                        | Ariana Grande
 Never Really Over                                                              | Katy Perry
 fuck, i'm lonely (with Anne-Marie) - from \u009313 Reasons Why: Season 3\u0094 | Lauv
 You Need To Calm Down                                                          | Taylor Swift
 Shallow                                                                        | Lady Gaga
(8 rows)

This is a window function that takes the average danceability for each genre and also displays the most common rating for danceability. 

SELECT genre, AVG(danceability), mode() WITHIN GROUP (ORDER BY danceability)
FROM sebmb.spot_df
WHERE genre IN ('dance pop','pop','latin','canadian hip hop','edm')
GROUP BY genre;

      genre       |         avg         | mode
------------------+---------------------+------
 canadian hip hop | 80.0000000000000000 |   73
 dance pop        | 70.2500000000000000 |   77
 edm              | 52.3333333333333333 |   29
 latin            | 72.0000000000000000 |   64
 pop              | 68.4285714285714286 |   48
(5 rows)

This is an aggregate function that takes the correlation between beats per minute and danceability for each genre. This one shows that edm has a high correlation between beats per minute and danceability for that genre. This could indicate that edm songs are good for fast paced dancing while latin might not meet that criteria. 

SELECT genre, COUNT(*), corr(bpm,danceability) as high_energy
FROM sebmb.spot_df
GROUP BY genre
ORDER BY COUNT(*) DESC;

      genre       | count |    high_energy
------------------+-------+--------------------
 dance pop        |     8 | -0.701518623828858
 pop              |     7 |  0.282034632085514
 latin            |     5 | -0.146809161792439
 canadian hip hop |     3 |  0.676131452661663
 edm              |     3 |  0.970725343394151
 reggaeton        |     2 |                 -1
 dfw rap          |     2 |                 -1
 country rap      |     2 |                 -1
 brostep          |     2 |                  1
 canadian pop     |     2 |                 -1
 panamanian pop   |     2 |
 reggaeton flow   |     2 |                 -1
 electropop       |     2 |
 pop house        |     1 |
 boy band         |     1 |
 australian pop   |     1 |
 trap music       |     1 |
 atl hip hop      |     1 |
 big room         |     1 |
 escape room      |     1 |
 r&b en espanol   |     1 |
(21 rows)